In [1]:
import folium
import pandas as pd
import numpy as np
import osmnx as ox
from shapely.geometry import Point


In [2]:
graph = ox.io.load_graphml("C:/Users/keyas/OneDrive/Documents/sih/map_matching_research/data/merged_graph.graphml")
# graph = ox.convert.to_undirected(graph)

In [3]:
nodes, edges = ox.graph_to_gdfs(graph)

In [4]:
for month in ['avg_visibility_January', 'avg_visibility_February', 'avg_visibility_March',
              'avg_visibility_April', 'avg_visibility_May', 'avg_visibility_June',
              'avg_visibility_July', 'avg_visibility_August', 'avg_visibility_September',
              'avg_visibility_October', 'avg_visibility_November', 'avg_visibility_December']:
    nodes[month] = pd.to_numeric(nodes[month], errors='coerce')
season_mapping = {
    'winter': ['November', 'December', 'January', 'February'],
    'spring': ['March', 'April', 'May'],
    'summer': ['June', 'July', 'August'],
    'autumn': ['September', 'October', 'November']
}


In [5]:
print(nodes.columns)

Index(['y', 'x', 'street_count', 'avg_visibility_January',
       'avg_rainfall_January', 'avg_cloud_cover_January',
       'avg_visibility_February', 'avg_rainfall_February',
       'avg_cloud_cover_February', 'avg_visibility_March',
       'avg_rainfall_March', 'avg_cloud_cover_March', 'avg_visibility_April',
       'avg_rainfall_April', 'avg_cloud_cover_April', 'avg_visibility_May',
       'avg_rainfall_May', 'avg_cloud_cover_May', 'avg_visibility_June',
       'avg_rainfall_June', 'avg_cloud_cover_June', 'avg_visibility_July',
       'avg_rainfall_July', 'avg_cloud_cover_July', 'avg_visibility_August',
       'avg_rainfall_August', 'avg_cloud_cover_August',
       'avg_visibility_September', 'avg_rainfall_September',
       'avg_cloud_cover_September', 'avg_visibility_October',
       'avg_rainfall_October', 'avg_cloud_cover_October',
       'avg_visibility_November', 'avg_rainfall_November',
       'avg_cloud_cover_November', 'avg_visibility_December',
       'avg_rainfall_Decembe

In [6]:
def calculate_seasonal_visibility(node_df, season_mapping):
    for season, months in season_mapping.items():
        visibility_columns = [f'avg_visibility_{month}' for month in months]
        node_df[f'avg_visibility_{season}'] = node_df[visibility_columns].mean(axis=1)
    return node_df

In [7]:
nodes = calculate_seasonal_visibility(nodes, season_mapping)

In [8]:
def visibility_to_color(visibility, min_visibility, max_visibility):
    if pd.isna(visibility):  
        return '#FFFFFF'  # white for missing data
    normalized_visibility = (visibility - min_visibility) / (max_visibility - min_visibility)
    colormap = folium.LinearColormap(['#ff0000', '#ffff00'], vmin=0, vmax=1)
    return colormap(normalized_visibility)

In [9]:
for season in season_mapping.keys():
    visibility_column = f'avg_visibility_{season}'
    min_visibility = nodes[visibility_column].min()
    max_visibility = nodes[visibility_column].max()
    
    print(f"Generating heatmap for {season} (min: {min_visibility}, max: {max_visibility})")

    m = folium.Map(location=[nodes['y'].mean(), nodes['x'].mean()], zoom_start=12)
    
    for node in nodes.index:
        visibility = nodes.loc[node, visibility_column]
        if pd.isna(visibility):
            continue
        
        lat, lon = nodes.loc[node, ['y', 'x']]
        color = visibility_to_color(visibility, min_visibility, max_visibility)
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"Node: {node}, Visibility: {visibility:.2f} mm"
        ).add_to(m)
    
    file_name = f"{season}_visibility_heatmap.html"
    m.save(file_name)
    print(f"Heatmap for {season} saved as {file_name}")
    m

Generating heatmap for winter (min: 1.4296647172353683, max: 13.168828097446646)
Heatmap for winter saved as winter_visibility_heatmap.html
Generating heatmap for spring (min: 3.9472811080027217, max: 13.798242286623102)
Heatmap for spring saved as spring_visibility_heatmap.html
Generating heatmap for summer (min: 2.4463975694157827, max: 14.373184310606831)
Heatmap for summer saved as summer_visibility_heatmap.html
Generating heatmap for autumn (min: 3.0527767438915863, max: 12.823781085272424)
Heatmap for autumn saved as autumn_visibility_heatmap.html
